In [ ]:
# default_exp scheduling
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import FRED
if torch.__version__[:4] == '1.13': # If using pytorch with MPS, use Apple silicon GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.has_mps else "cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
%load_ext autoreload
%autoreload 2

# 04a Loss Scheduling Utils

# Sigmoid Scheduler

In [ ]:
# # export
# def general_scheduler(oss_weights:dict, loss_name:str, epoch:int, total_epochs:int,  c:int=1, function_type:str="sigmoid"):
#     '''
#     Inputs
#     loss_weights: Dictionary of loss names and their weights
#     loss_name: the loss in the loss_weights to be scheduled
#     epoch: current epoch
#     total_epochs: total number of epochs that will be run
#     c: scaler multiple for sigmoid, or slope for linear
#     function_type: type of function for the scheduler, can be 
#         "sigmoid": sigmoid 1 / (1 + np.exp(c(-epoch+np.floor(total_epochs/2))))
#         "off": 1 - "sigmoid"
#         "linear": linear with slope c
#         "loglinear": log linear with weight c
    
#     Return
#     loss_weights dictionary with updated weight for the specified loss function at the current epoch
#     '''
#     if function_type == "sigmoid":
#         new_weight = 1 / (1 + np.exp(-epoch+np.floor(total_epochs/2)))
#     elif function_type == "off"
#         new_weight = 1 - (1 / (1 + np.exp(-epoch+np.floor(total_epochs/2))))
#     loss_weights[loss_name] = new_weight
#     elif function_type == "linear"
#         new_weight = c*epoch
#     elif function_type == "loglinear"
#         new_weight = np.exp(c*epoch)
#     return loss_weights  
    

In [ ]:
# # export
# def specific_scheduler(Dictionary loss_weights, int epoch):
#     loss_weights = sigmoid_scheduler(loss_weights, loss_name="smoothness", epoch, total_epochs=10, c=1, function_type="sigmoid")
#     # change parameters manually, however you'd like:
#     # loss_weights = sigmoid_scheduler(loss_weights, loss_name="flow neighbor loss", epoch, total_epochs=20, c=5, function_type="off")
#     # loss_weights = sigmoid_scheduler(loss_weights, loss_name="distance regularization", epoch, total_epochs=20, c=1, function_type="off")
#     return loss_weights

In [ ]:
# export
class block_scheduler():
    def __init__(
            self,
            epoch_to_transition,
            loss_to_turn_on,
            weight = 1
    ):
        self.epoch_number = 0
        self.epoch_to_transition = epoch_to_transition
        self.loss_to_turn_on = loss_to_turn_on
        self.loss_weight = weight
    def __call__(self,loss_weights):
        self.epoch_number += 1
        if self.epoch_number > self.epoch_to_transition:
            loss_weights[self.loss_to_turn_on] = self.loss_weight
        else:
            if self.epoch_number > 1:
                loss_weights[self.loss_to_turn_on] = 0
        return loss_weights